In [119]:
import time
import pandas as pd

In [120]:
import io_utils


def isolate_lineage(node, data):
    lineage = [node]
    
    for n in data:
        if len(n) == len(node):
            continue
        if set(n).issubset(set(node)):
            lineage.append(n)
        if set(node).issubset(set(n)):
            lineage.append(n)
    
    return lineage


def break_by_lineage(data):
    bySize = dict()

    for node in data:
        size = len(node)
        bySize.setdefault(size, []).append(node)

    lineage_lines = []
    remaining = set(data)
    
    while len(remaining) > 0:
        next_key = sorted(bySize.keys())[-1]
        next_layer = set(bySize[next_key]).intersection(remaining)
        bySize.pop(next_key)

        for node in next_layer:
            lineage = isolate_lineage(node, data)
            lineage_lines.append(lineage)
            remaining -= set(lineage)

    return lineage_lines

In [121]:
data = io_utils.import_data('../data/6.txt')
print(break_by_lineage(data))

[[(1, 2, 3, 4, 5), (2,), (1, 2), (2, 3), (1, 2, 3), (1, 2, 3, 4)]]


In [122]:
dataset = '3515'
data = io_utils.import_data(f'../data/{dataset}.txt')
t0 = time.time()
chunks = break_by_lineage(data)
print(f"Time: {time.time() - t0}")

Time: 4.444952964782715


# Success?

In [123]:
known_nodes = set()
doubled_nodes = set()

for chunk in chunks:
    for node in chunk:
        if node in known_nodes:
            doubled_nodes.add(node)
        known_nodes.add(node)

print(f"Of the {len(known_nodes)} nodes, {len(doubled_nodes)} are doubled")
print(f"{len(known_nodes) - len(doubled_nodes)} are unique")
print(f"\tOverlap: {len(doubled_nodes) * 100 / len(known_nodes)}%")

print(f"{len(chunks)} chunks")
df = pd.DataFrame([len(chunk) for chunk in chunks])
df.describe()

Of the 3515 nodes, 2939 are doubled
576 are unique
	Overlap: 83.61308677098151%
328 chunks


,0
count,328.000000
mean,423.570122
std,610.143184
min,3.000000
25%,31.750000
50%,165.000000
75%,352.500000
max,1999.000000
